In [1]:
import torch
import sys
import json
from transformers import BertTokenizer, BertForSequenceClassification
from peft import PeftModel, PeftConfig
from models import CustomTokenizer

/home/korenikil/laba-dla-lechi/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
def predict(text, model, tokenizer, device, max_length=128, threshold=0.8):
    model.eval()
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        probs = torch.softmax(logits, dim=1)
        pred_class = torch.where(probs[:, 1] > threshold, 1, 0)
        confidence = probs[0][pred_class].item()
    return pred_class.item(), confidence

In [3]:
print("Загрузка конфигурации модели...")
with open("model_config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = CustomTokenizer(config["bert_model_name"])

peft_model_dir = "concert_classifier_peft"

peft_config = PeftConfig.from_pretrained(peft_model_dir)
base_model = BertForSequenceClassification.from_pretrained(
    peft_config.base_model_name_or_path,
    num_labels=config["num_classes"]
)
model = PeftModel.from_pretrained(base_model, peft_model_dir)
model.to(device)
model.eval()

Загрузка конфигурации модели...
Loaded 1529744 words from custom dictionary


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (defaul

In [ ]:
positive_queries = [
    "Хочу купить билеты на концерт группы Кино",
    "Мне нужно два билета на выступление Арии",
    "Где можно приобрести билеты на Radiohead?",
    "Ищу билеты на концерт The Beatles в Москве",

    "Хочу сходить на концерт в эту пятницу",
    "Мне нужны билеты на любой рок-концерт",
    "Где найти билеты на ближайший концерт?",
    "Хочу посетить музыкальное мероприятие",

    "Хочу на концерт группы ЛунныеКотики",
    "Билеты на выступление Звездопад нужны",
    "Ищу концерт группы Мечтатели2024",
    
    "Можно ли купить билеты на концерт?",
    "Ищу где послушать живую музыку",
    "Хотел бы посетить выступление артиста",
    "Нужны два места на концерт",
    
    "Хочу на концерт в следующем месяце",
    "Билеты на завтрашнее выступление нужны",
    "Ищу концертные билеты на выходные",

    "Хочу купить билеты на концерт в СПБ",
    "Ищу выступление в моем городе",
    "Концерт в Москве, хочу пойти",

    "Классно было бы сходить на концерт!",
    "Может, сходим на какой-нибудь концерт?",
    "Очень хочется послушать живую музыку"
]

In [ ]:
negative_queries = [
    "Хочу купить новый телефон",
    "Мне нужна помощь с компьютером",
    "Где найти рецепт борща?",
    "Как добраться до вокзала?",

    "Скачай мне эту песню",
    "Какой альбом группы Кино лучший?",
    "Ищу текст песни Арии",
    "Хочу послушать музыку онлайн",

    "Хочу в кино на новый фильм",
    "Билеты в театр на выходные",
    "Иду на выставку картин",

    "Куплю билеты на самолет",
    "Нужны билеты в аквапарк",
    "Где купить билеты в цирк?",
    
    "Люблю музыку группы Кино",
    "The Beatles - моя любимая группа",
    "Слушаю Radiohead каждый день",
    
    "Я люблю музыку",
    "Концерты - это весело",
    "Хорошая погода сегодня",
    
    "Что такое концерт?",
    "Когда был последний концерт Кино?",
    "Сколько стоит билет на концерт?",

    "Вчера был на концерте",
    "Видел выступление Арии",
    "Ходил на концерт Radiohead"
]

In [22]:
text = "The Beatles - моя любимая группа"

pred, confidence = predict(text, model, tokenizer, device)
label = "Желание посетить концерт" if pred == 1 else "Нет желания посетить концерт"
print(f"Текст: {text}")
print(f"Предсказание: {label}")
print(f"Уверенность: {confidence:.4f}")

Текст: The Beatles - моя любимая группа
Предсказание: Нет желания посетить концерт
Уверенность: 0.4610
